# Mashup of datasets of air quality and cars data of Bologna

The dataset is the product of  mashup of four datasets published by the city council of Bologna. The four datasets are:

* Bologna air quality
* Statistics on cars of the Bologna residents
* Data on the public transport fleet of Bologna
* Number of passengers of public transport in the city of bologna

The data covers years from 2016 to 2024. 

### Data fields

* *DATE*: the date in the year-month format
* *C6H6 (BENZENE)*: air pollutants
* *CO (MONOSSIDO DI CARBONIO)*: air pollutants
* *NO (MONOSSIDO DI AZOTO)*: air pollutants
* *NO2 (BIOSSIDO DI AZOTO)*: air pollutants
* *NOX (OSSIDI DI AZOTO)*: air pollutants
* *PM10*: air pollutants
* *PM2.5*: air pollutants
* *TOTAL VEHICLES*: The total number of private vehicles owned by the residents of Bologna
* *BENZINA*: Private vehicles with petrol power supply
* *BENZINA E GAS LIQUIDO*: Private vehicles with petrol and liquid gas power supply
* *BENZINA E METANO*: Private vehicles with petrol and methane power supply
* *ELETTRICITA*: Private vehicles with Electric power supply
* *GASOLIO*: Private vehicles with gasoline power supply
* *IBRIDO BENZINA*: Private vehicles with hybrid petrol power supply
* *IBRIDO GASOLIO*: Private vehicles with  hybrid gasoline power supply
* *METANO*: Private vahicles with methane power supply
* *Diesel_Ibrido*: Public transport with hybrid diesel power supply
* *Diesel_NON ibrido*: Public transport with non-hybrid diesel power supply
* *Elettrico_NON ibrido*: Public transport with electric non-hybrid power supply
* *Gas naturale_Ibrido*:  Public transport with hybrid natural gas power supply
* *Gas naturale_NON ibrido*:  Public transport with non-hybrid natural gas power supply
* *TOTAL PUBLIC FLEET*: Total number of public transoprt fleet
* *URBAN PASSENGERS BOLOGNA*: Total number of passengers of public transport in Bologna


## Python code for data cleaning and mashup

In [22]:
import pandas as pd
from functools import reduce

#**Functions**
#This function is used to build and transform dataframes. It is used in working with three datasets.

def build_dataframe(df, div=None):
    df.replace(r"(?<=\d)\.(?=\d)", "", inplace=True, regex = True) # Remove dots from numbers
    matching_year = df.columns[df.columns.str.contains('anno', case=False)].tolist()# Extract the year from the dataset 
    strt = df.loc[0,matching_year[0]]
    en = df.loc[len(df)-1,matching_year[0]]
    
    # Build the month period for the year
    dates = pd.period_range(start=f'{strt}-01', end=f'{en}-12', freq='M', name='DATE').astype('str') 
    dates = pd.Series(dates)
    agg_df = pd.concat([dates, df], ignore_index=False).reset_index()
    agg_df.drop([matching_year[0], 'index'], axis=1, inplace=True)
    agg_df = agg_df.iloc[:-len(df)]
        
    # Populate dataframe
    # Add a "year" column to dataset for comparison of the year
    agg_df['year'] = pd.to_datetime(agg_df['DATE']).dt.year
    for idx, row in agg_df.iterrows():
        year = row['year']
        for name in agg_df.columns[1:-1]:  # skip first (date) and last (new year)
            # Filter vcl_bol_sum where "Anno" matches year
            matching_row = df.loc[df[matching_year[0]] == year, name].astype('int32')
            if not matching_row.empty and div==None:
                agg_df.at[idx, name] = matching_row.values[0]
            elif not matching_row.empty and div:
                agg_df.at[idx, name] = round(matching_row.values[0] / div)
    agg_df.drop('year', axis=1, inplace=True)
    return agg_df

## Read and Clean Bologna air quality dataset

The dataset is a large dataset. So first we downloaded it and then read it in the jupyter-notebook instead of reading the dataset directly from the URL, to prevent waste of time everytime we run the notebook.
The following code is the steps in which we read the dataset, normalize it, groupby and aggregate the data with mean.
Normalizing measures:
- Change the all the mg/m3 units to μg/m3 to standarze the units
- Remove NaN value rows
- Remove unnecessary columns
- Aggregagate the data by year and month. because the dataset is hourly recorded. we took the mean for this purpos
- put the pollutant agents in seperate columns and put the corresponding value in the field
- Round the floating point number to 2 decimals

For this study we did not consider different stations of the City seperately. We took the arithmetic average of the three stations.


In [7]:
first_ds = "../original-datasets/ds1.csv"

air_quality = pd.read_csv(first_ds,
                            sep  = ';',
                            parse_dates = ['DATA_INIZIO', 'DATA_FINE'],
                            dtype = 'str'
                         )

# Converting the date column to datetime
air_quality['DATA_INIZIO'] = pd.to_datetime(air_quality['DATA_INIZIO'], utc=True)

# Drop empty rows
air_quality = air_quality.dropna(subset=['DATA_INIZIO'])
air_quality = air_quality.dropna(subset=['VALORE'])

# Convert datetime to year-month period
air_quality['DATE'] = air_quality['DATA_INIZIO'].dt.to_period('M').astype('str')

/tmp/ipykernel_8994/637464795.py:17: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  air_quality['DATE'] = air_quality['DATA_INIZIO'].dt.to_period('M').astype('str')


In [8]:
# Convert value collumn to float
air_quality['VALORE'] = air_quality['VALORE'].astype(float)

# Convert mg to ug in UM column
air_quality.loc[air_quality['UM'] == 'mg/m3', 'VALORE'] *= 1000
air_quality['UM'] = 'ug/m3'  # Set all units to ug

# Drop unneeded columns
air_quality.drop(["COD_STAZ", "DATA_INIZIO", "DATA_FINE"], axis=1, inplace=True)

In [9]:
# Create pivot table for groupby pollutant agents in the dataset and by month
air_quality_pivot = air_quality.pivot_table(
                        index = 'DATE',
                        columns = 'AGENTE',
                        values = 'VALORE',
                        aggfunc = 'mean'
        ).reset_index()


* Here we delete O3 because it does not a direct product of vehicles


In [10]:
air_quality_pivot.drop("O3 (OZONO)", axis=1, inplace=True)
air_quality_pivot.drop(air_quality_pivot.index[[0,-1]], inplace=True) # Drop 2016-12 and 2025-01 records

# Round the floating numbers to two 
air_bol = air_quality_pivot.round(2)

air_bol.head()

AGENTE,DATE,C6H6 (BENZENE),CO (MONOSSIDO DI CARBONIO),NO (MONOSSIDO DI AZOTO),NO2 (BIOSSIDO DI AZOTO),NOX (OSSIDI DI AZOTO),PM10,PM2.5
1,2017-01,2.30,828.23,45.07,54.65,138.24,58.49,48.06
2,2017-02,2.53,965.74,40.45,45.70,116.95,36.82,30.93
3,2017-03,1.39,789.87,21.35,33.45,83.65,26.38,18.86
4,2017-04,1.01,752.30,11.24,24.00,58.36,16.99,10.29
5,2017-05,0.96,794.96,12.07,23.09,59.77,13.90,8.65


## Read and clean the Private Vehicles dataset in Bologna

The dataset contains several different metrics regarding private auromobiles in Bologna. We only selected the data that are useful and is related to air quality.

In [11]:
second_ds = "../original-datasets/ds2.csv"

vehicles_bol = pd.read_csv(second_ds,
                         sep = ';'
                          )

Fill the missing values with 0, and selecting the columns we need for our study. After that we do the groupby year. because we do not need different postcode zones of Bologna.

In [12]:
vehicles_bol.replace('', 0, inplace=True)
vehicles_bol.replace('ND', 0, inplace=True)

vehicles_bol.fillna(0, inplace=True)

# Change the format of all the fields to integer
vehicles_bol = vehicles_bol.astype(int)

# Needed columns
columns = ['TOTAL VEHICLES', 'BENZINA', 'BENZINA E GAS LIQUIDO',
       'BENZINA E METANO', 'ELETTRICITA', 'GASOLIO', 'IBRIDO BENZINA',
       'IBRIDO GASOLIO', 'METANO']

vehicles_bol.rename({'totale_veicoli':'TOTAL VEHICLES'}, axis=1, inplace=True)

vcl_bol_sum = vehicles_bol.groupby('Anno')[columns].sum().reset_index()

vcl_bol = build_dataframe(vcl_bol_sum).astype('int32', errors='ignore')

vcl_bol.head()

,DATE,TOTAL VEHICLES,BENZINA,BENZINA E GAS LIQUIDO,BENZINA E METANO,ELETTRICITA,GASOLIO,IBRIDO BENZINA,IBRIDO GASOLIO,METANO
0,2019-01,208487,95927,22858,13310,191,68863,7095,231,0
1,2019-02,208487,95927,22858,13310,191,68863,7095,231,0
2,2019-03,208487,95927,22858,13310,191,68863,7095,231,0
3,2019-04,208487,95927,22858,13310,191,68863,7095,231,0
4,2019-05,208487,95927,22858,13310,191,68863,7095,231,0


## Read and clean urban public transport fleet of Bologna

In [13]:
third_ds = "../original-datasets/ds3.csv"

pub_fleet = pd.read_csv(third_ds,
                        encoding="utf-16", # The dataset is utf-16 encoded
                        sep = '\t',
                        header=[0,1]
                       )

pub_fleet.columns = pub_fleet.columns.map('_'.join)

pub_fleet.rename(columns={'Unnamed: 0_level_0_Anno':'Anno', 
                          'Totale_Totale':'TOTAL PUBLIC FLEET'}, inplace=True)

pub_fleet.fillna(0, inplace=True)

pub_bol = build_dataframe(pub_fleet).astype('int32', errors='ignore')

pub_bol.head()

,DATE,Diesel_Ibrido,Diesel_NON ibrido,Elettrico_NON ibrido,Gas naturale_Ibrido,Gas naturale_NON ibrido,TOTAL PUBLIC FLEET
0,2012-01,40,219,59,1,148,467
1,2012-02,40,219,59,1,148,467
2,2012-03,40,219,59,1,148,467
3,2012-04,40,219,59,1,148,467
4,2012-05,40,219,59,1,148,467


## Read and clean passengers dataset

The dataset contains data on other "comune"s in the Municipality of Bologna. But because of the locations of the stations of controlling air quality in the the air quality dataset, we decided to consider only the Bologna passengers and neglect interurban, and other "comune", presented in the dataset.

In [15]:
forth_ds ="../original-datasets/ds4.csv"

passengers = pd.read_csv(forth_ds,
                        encoding="utf-16", # The dataset is utf-16 encoded
                        sep = '\t',
                        header=[0,1]
                       )

passengers.columns = passengers.columns.map('_'.join)

passengers.drop(passengers.columns[2:], axis=1, inplace=True)

passengers.rename(columns={'Unnamed: 0_level_0_Anno':'Anno', 
                           'Servizio Urbano_Bologna':'URBAN PASSENGERS BOLOGNA'}, inplace=True)

pass_bol = build_dataframe(passengers,12)

pass_bol["URBAN PASSENGERS BOLOGNA"] = pass_bol["URBAN PASSENGERS BOLOGNA"].astype('int32', errors="ignore")

pass_bol.head()

,DATE,URBAN PASSENGERS BOLOGNA
0,2016-01,9103277
1,2016-02,9103277
2,2016-03,9103277
3,2016-04,9103277
4,2016-05,9103277


## The mashup

The following is the mashup of all four datasets.

In [19]:
dataframes = [air_bol, vcl_bol, pub_bol, pass_bol]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['DATE'],
                                            how='left'), dataframes)

df_merged.head()

,DATE,C6H6 (BENZENE),CO (MONOSSIDO DI CARBONIO),NO (MONOSSIDO DI AZOTO),NO2 (BIOSSIDO DI AZOTO),NOX (OSSIDI DI AZOTO),PM10,PM2.5,TOTAL VEHICLES,BENZINA,...,IBRIDO BENZINA,IBRIDO GASOLIO,METANO,Diesel_Ibrido,Diesel_NON ibrido,Elettrico_NON ibrido,Gas naturale_Ibrido,Gas naturale_NON ibrido,TOTAL PUBLIC FLEET,URBAN PASSENGERS BOLOGNA
0,2017-01,2.30,828.23,45.07,54.65,138.24,58.49,48.06,NaN,NaN,...,NaN,NaN,NaN,36.0,165.0,101.0,0.0,206.0,508.0,9378950.0
1,2017-02,2.53,965.74,40.45,45.70,116.95,36.82,30.93,NaN,NaN,...,NaN,NaN,NaN,36.0,165.0,101.0,0.0,206.0,508.0,9378950.0
2,2017-03,1.39,789.87,21.35,33.45,83.65,26.38,18.86,NaN,NaN,...,NaN,NaN,NaN,36.0,165.0,101.0,0.0,206.0,508.0,9378950.0
3,2017-04,1.01,752.30,11.24,24.00,58.36,16.99,10.29,NaN,NaN,...,NaN,NaN,NaN,36.0,165.0,101.0,0.0,206.0,508.0,9378950.0
4,2017-05,0.96,794.96,12.07,23.09,59.77,13.90,8.65,NaN,NaN,...,NaN,NaN,NaN,36.0,165.0,101.0,0.0,206.0,508.0,9378950.0


## Output CSV

In [21]:
df_merged.to_csv("../mashup-datasets/ds_bologna.csv", index=False, encoding="utf-8")